In [3]:
import numpy as np
import pandas as pd

## pandas

In [52]:
df = pd.DataFrame({ 'A' : 1.,
                    'B' : pd.Timestamp('20130102'),
                    'C' : pd.Series(1,index=list(range(4)),dtype='float32'),
                    'D' : np.array([3] * 4,dtype='int32'),
                    'E' : pd.Categorical(["test","train","test","train"]),
                    'F' : 'foo',
                    'G' : ['1000', '', '123', '1234',]})

In [53]:
df

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1000
1,1.0,2013-01-02,1.0,3,train,foo,
2,1.0,2013-01-02,1.0,3,test,foo,123
3,1.0,2013-01-02,1.0,3,train,foo,1234


## NaN処理

In [54]:
# NaNであるか
df.isna()

,A,B,C,D,E,F,G
0,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False


In [55]:
# 全項目NaN化
df[df.isna()]

,A,B,C,D,E,F,G
0,NaN,NaT,NaN,NaN,NaN,NaN,NaN
1,NaN,NaT,NaN,NaN,NaN,NaN,NaN
2,NaN,NaT,NaN,NaN,NaN,NaN,NaN
3,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [56]:
# blankがNaN
df[~df.isna()]

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1000
1,1.0,2013-01-02,1.0,3,train,foo,
2,1.0,2013-01-02,1.0,3,test,foo,123
3,1.0,2013-01-02,1.0,3,train,foo,1234


## replace NaN

In [57]:
# EX) df.replace([-1, 999, 1000], np.nan)
df = df.replace('', np.nan)
df

,A,B,C,D,E,F,G
0,1.0,2013-01-02,1.0,3,test,foo,1000
1,1.0,2013-01-02,1.0,3,train,foo,NaN
2,1.0,2013-01-02,1.0,3,test,foo,123
3,1.0,2013-01-02,1.0,3,train,foo,1234


## NaNを0,000に変換

In [58]:
df[df['G'].isnull()].replace(np.nan, '0,000')

,A,B,C,D,E,F,G
1,1.0,2013-01-02,1.0,3,train,foo,"0,000"


# 関数処理まとめ

## 価格表記に変換

In [128]:
df = pd.DataFrame({ 'A' : 1.,
                    'G' : ['1000', np.nan, '123', '1234', '12345', 123456]})

In [129]:
df

,A,G
0,1.0,1000
1,1.0,NaN
2,1.0,123
3,1.0,1234
4,1.0,12345
5,1.0,123456


In [153]:
def rep_digit_num(df, before_col='code', after_col='L_code', digit=5):
    """
	値段表記に変換する
    
    ExSample:
    	123.0  ==>> 123
        1234.0 ==>> 1,234
        NaN	   ==>> 0,000
    
    params:
    	df(DataFrame)	: 処理してるデータ
        before_col(str) : 処理対象col
        after_col(str)  : 処理したcol
        
    
    return:
    	after_col(np.str)digit桁にした値を返す
    """
    filter_ = df[~df[before_col].isna()]
    filter_ = filter_[before_col].astype(np.int)
    
    df[after_col] = filter_.apply(lambda row: f"{row:,}".replace('NaN', '0,000'))
#     df[after_col] = df.apply(lambda row: f"{row[before_col]:,}", axis=1).replace('nan', '0,000', regex=True)
    
    return df

In [154]:
rep_digit_num(df, before_col='G', after_col='price')

,A,G,G_L_num,price
0,1.0,1000,"1,000","1,000"
1,1.0,NaN,NaN,NaN
2,1.0,123,123,123
3,1.0,1234,"1,234","1,234"
4,1.0,12345,"12,345","12,345"
5,1.0,123456,"123,456","123,456"


In [142]:
"{:,}".format(12345678)

'12,345,678'

In [143]:
i = 12345678
f"{i:,}"

'12,345,678'

In [155]:
str_ = '12345678'
f"{str_:,}"

ValueError: Cannot specify ',' with 's'.